In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import folium

data = {
    'Швидкість': np.random.uniform(30, 120, 1000),
    'Дорожні_умови': np.random.choice(['Сухо', 'Мокро', 'Сніг'], 1000, p=[0.7, 0.2, 0.1]),
    'Видимість': np.random.uniform(0.1, 10, 1000),
    'Ризик_аварії': np.zeros(1000)
}
df = pd.DataFrame(data)

df['Ризик_аварії'] = np.where(
    (df['Швидкість'] > 90) | (df['Дорожні_умови'].isin(['Мокро', 'Сніг'])),
    1, 0
)
df['Дорожні_умови_кодовані'] = df['Дорожні_умови'].astype('category').cat.codes

print(df.head())
features = ['Швидкість', 'Видимість', 'Дорожні_умови_кодовані']
target = 'Ризик_аварії'

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"\nТочність моделі: {accuracy:.2f}")

feature_importances = pd.Series(model.feature_importances_, index=features).sort_values(ascending=False)
print("\nВажливість ознак:")
print(feature_importances)
df_map = pd.DataFrame({
    'lat': np.random.uniform(34.0, 34.5, 20),
    'lon': np.random.uniform(-118.5, -118.0, 20),
    'Швидкість': np.random.uniform(30, 120, 20),
    'Дорожні_умови': np.random.choice(['Сухо', 'Мокро'], 20, p=[0.8, 0.2]),
    'Видимість': np.random.uniform(0.1, 10, 20), # Added 'Видимість' column
})

df_map['Дорожні_умови_кодовані'] = df_map['Дорожні_умови'].astype('category').cat.codes
df_map['Прогноз_ризику'] = model.predict(df_map[features])

m = folium.Map(location=[34.05, -118.25], zoom_start=11)

for _, row in df_map.iterrows():
    color = 'red' if row['Прогноз_ризику'] == 1 else 'green'
    tooltip = f"Швидкість: {row['Швидкість']:.1f} км/год<br>Умови: {row['Дорожні_умови']}<br>Ризик: {'Високий' if row['Прогноз_ризику'] == 1 else 'Низький'}"
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        tooltip=tooltip
    ).add_to(m)

m

    Швидкість Дорожні_умови  Видимість  Ризик_аварії  Дорожні_умови_кодовані
0   96.016218          Сухо   3.659347             1                       2
1  116.820518         Мокро   4.795164             1                       0
2   41.973951          Сухо   8.692280             0                       2
3   47.791284          Сухо   4.775129             0                       2
4   97.097609          Сніг   2.421876             1                       1

Точність моделі: 1.00

Важливість ознак:
Швидкість                 0.498780
Дорожні_умови_кодовані    0.471404
Видимість                 0.029817
dtype: float64
